In [ ]:
import cyautomation.cyschoolhouse as cysh

## Daily

In [ ]:
## sync enrollment in DESSA sections
results1 = cysh.student_section.enrollment_sync(
    source_section = 'Tutoring: Math',
    destination_section = 'DESSA',
    enrollment_start_date = '2018-09-05',
    ACM_to_TL = True
)

results2 = cysh.student_section.enrollment_sync(
    source_section = 'Tutoring: Literacy',
    destination_section = 'DESSA',
    enrollment_start_date = '2018-09-05',
    ACM_to_TL = True
)

results3 = cysh.student_section.enrollment_sync(
    source_section = 'Coaching: Attendance',
    destination_section = 'DESSA',
    enrollment_start_date = '2018-09-05',
    ACM_to_TL = True
)

# theoretically redundant
results4 = cysh.student_section.enrollment_sync(
    source_section = 'SEL Check In Check Out',
    destination_section = 'DESSA',
    enrollment_start_date = '2018-09-05',
    ACM_to_TL = True
)

## sync enrollment in MIRI sections
results5 = cysh.student_section.enrollment_sync(
    source_section = 'Tutoring: Math',
    destination_section = 'Math Inventory',
    enrollment_start_date = '2018-09-05'
)

results6 = cysh.student_section.enrollment_sync(
    source_section = 'Tutoring: Literacy',
    destination_section = 'Reading Inventory',
    enrollment_start_date = '2018-09-05'
)

## As Needed/Requested

#### Add New Students to cyschoolhouse

In [ ]:
## Requires salesforce 2FA
cysh.student.upload_all(
    xlsx_dir = 'Z:/ChiPrivate/Chicago Data and Evaluation/SY19', 
    xlsx_name = 'SY19 New Students for cyschoolhouse.xlsx', 
    enrollment_date='8/3/2018',
)

In [ ]:
#from time import sleep
#sleep(60)
cysh.student.update_student_External_Id()

#### Indicator Area Enrollment

In [ ]:
# load students to be assigned
df = cysh.ia_assignment_chi.get_ia_to_assign()
print(len(df), 'student IAs to enroll')

In [ ]:
# load browser
ia = cysh.ia_assignment_chi.IndicatorAreaEnrollment()

In [ ]:
# Enroll - Requires salesforce 2FA
ia.enroll_all_students(max_errors=100)

In [ ]:
# cysh.tracker_mgmt.unprotect_ACM_validation_sheet(resource_type='SY19 Leadership Tracker', containing_folder='Leadership Team Documents')
# cysh.tracker_mgmt.unprotect_ACM_validation_sheet(resource_type='SY19 Coaching Log', containing_folder='Leadership Team Documents')

## Weekly

In [ ]:
# Run when new ACMs start
trackers = cysh.CoachingLog(test=True)
trackers.update_all_acm_stdnt_validation_sheets()

In [ ]:
trackers = cysh.LeadershipTracker(test=True)
trackers.update_all_acm_stdnt_validation_sheets()

In [ ]:
# Run when new ACMs start (or new students enrolled in attendance)
trackers = cysh.AttendanceTracker(test=True)
trackers.update_all_acm_stdnt_validation_sheets()

In [ ]:
## ToT Audit -- could be a Power BI report, but would need ACM access
result = cysh.tot_audit.fix_T1T2ELT()
tot_error_df = cysh.tot_audit.get_error_table()
cysh.tot_audit.write_error_tables_to_cyconnect(tot_error_df)

counts = tot_error_df.groupby(['School', 'Error'])['ACM'].count().reset_index(level='Error').rename(columns={'ACM':'Count'})
counts.to_excel('Z:/Impact Analytics Team/SY19 ToT Audit Error Counts.xlsx')

In [ ]:
## Service Trackers
cysh.servicetrackers.update_service_trackers()

## Start of Year and As Needed

In [ ]:
att_trackers = cysh.AttendanceTracker(test=True)
att_trackers.deploy_all()

In [ ]:
ldr_trackers = cysh.LeadershipTracker(test=True)
ldr_trackers.deploy_all()

In [ ]:
cch_trackers = cysh.CoachingLog(test=True)
cch_trackers.deploy_all()

In [ ]:
## Create Sections -- Requires salesforce 2FA
df = cysh.section_creation_chi.academic_sections_to_create()
df

In [ ]:
cysh.section_creation.section_creation?

In [ ]:
df = cysh.section_creation_chi.non_CP_sections_to_create()
df

In [ ]:
cysh.section_creation.section_creation()

In [ ]:
df = cysh.section_creation_chi.MIRI_sections_to_create()
df

In [ ]:
cysh.section_creation.section_creation()

In [ ]:
cysh.section_creation_chi.deactivate_all_sections(section_type='50 Acts of Greatness')

In [ ]:
## Distribute Trackers
#cysh.tracker_mgmt.deploy_tracker(resource_type='SY19 Attendance Tracker', containing_folder='Team Documents')

## End of Year

In [ ]:
## Export salesforce Objects

## Exit all student_sections

# Misc.

In [ ]:
# # Associate TLs with their ACMs' SEL sections

# section_df = cysh.get_object_df('Section__c')
# section_df = section_df.loc[section_df['Name'].str.contains('SEL')]

# staff_df = cysh.get_object_df('Staff__c', ['Id', 'Name', 'Role__c', 'Organization__c'])
# staff_df = staff_df.loc[staff_df['Role__c'].str.contains('Team Lead')==True]

# staff_section_df = cysh.get_object_df('Staff_Section__c')

# results = []
# for index, row in section_df.iterrows():
#     if len(staff_section_df.loc[staff_section_df['Section__c']==row['Id']]) > 1:
#         print(f"ERROR: Section: {row['Id']} already has more than 1 staff associated")
#         continue
#     staff = staff_df.loc[staff_df['Organization__c']==row['School__c'], 'Id']
#     if len(staff) != 1:
#         print(f"ERROR: 0 or many Team Leaders associated with school: {row['School__c']}")
#         continue
#     staff_section_dict = {}
#     staff_section_dict['Account__c'] = row['School__c']
#     staff_section_dict['Section__c'] = row['Id']
#     staff_section_dict['Staff__c'] = staff.values[0]
#     staff_section_dict['Intervention_Primary_Staff__c'] = False
    
#     result = cysh.sf.Staff_Section__c.create(staff_section_dict)
#     results.append(result)

# sch_ref_df = pd.read_excel(r'Z:\ChiPrivate\Chicago Data and Evaluation\SY19\SY19 School Reference.xlsx')
# 
# tracker.fill_all_coaching_log_acm_rollup(sch_ref_df.loc[sch_ref_df['Informal Name']=='Chicago Academy'])